In [ ]:
import pandas as pd

df = pd.read_csv("forecasting_test.csv")
df.isnull().sum()

In [ ]:
"""
The last column represents the residence size, but we already got this information based on the hhsize columns, and since a lot of rows are missing we can drop it.
We got 6 hhsize and 8 income columns which are being one-hot-encoded. To handle easier the data for now we can undo this encoding.
Exactly 2150 rows are null, simultaneously, for zipcode, mozip, children, income and owner - which may happen when those may belong to a similar category.
The 2009 (luse1-6) missing values are always missing as a group. It doesn't happen for a single month of a house, therefore filling the data backward/forward may not be optimal.
We got luse1-6 which are just lusage but for the year 2009, but in the same time we got the lusage for 2010 and 2011 as rows (so we should either move them all as rows or make new columns per a hh).
"""

df = df.drop(columns=["size"])

# Merge the income columns into a single one - where the values here will be from 2 to 9
# e.g. if the value is 5 then it's like we had income5=1 and the other income columns =0
df['income'] = df.loc[:,"income2":"income9"].idxmax(axis=1).str[-1]
df['income'] = pd.to_numeric(df['income'], errors='coerce')
df = df.drop(columns=df.loc[:,"income2":"income9"])

# Merge the hhsize columns into a single one - where the values here will be from 2 to 6
df = df.rename(columns={'hhsize5plus': 'hhsize6'})
df['hhsize'] = df.loc[:,"hhsize2":"hhsize6"].idxmax(axis=1).str[-1]
df['hhsize'] = pd.to_numeric(df['hhsize'], errors='coerce')
df = df.drop(columns=df.loc[:,"hhsize2":"hhsize6"])

# Fill the missing values from static features
# It appears that all the columns where exactly 2150 rows got null values belong to hhsize5plus (hhsize=6)
missing_data_rows = df[df[['zipcode', 'mozip', 'children', 'income', 'owner']].isnull().any(axis=1)]
print(sum(missing_data_rows['hhsize'] != 6))

# So we'll fill the data with the most frequent value of each column (with the condition that this value belong to hhsize=6)
df.loc[df['hhsize'] == 6, 'zipcode'] = df.loc[df['hhsize'] == 6, 'zipcode'].fillna(df[df['hhsize'] == 6]['zipcode'].mode()[0])
df.loc[df['hhsize'] == 6, 'mozip'] = df.loc[df['hhsize'] == 6, 'mozip'].fillna(df[df['hhsize'] == 6]['mozip'].mode()[0])
df.loc[df['hhsize'] == 6, 'children'] = df.loc[df['hhsize'] == 6, 'children'].fillna(df[df['hhsize'] == 6]['children'].mode()[0])
df.loc[df['hhsize'] == 6, 'owner'] = df.loc[df['hhsize'] == 6, 'owner'].fillna(df[df['hhsize'] == 6]['owner'].mode()[0])
df.loc[df['hhsize'] == 6, 'income'] = df.loc[df['hhsize'] == 6, 'income'].fillna(df[df['hhsize'] == 6]['income'].mode()[0])

# Fill the missing lusage values (from 2009)
# A quick inspection shows that the lusage values dont have an immediate visual correlation like above, so we'll impute based on other features
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# lusage was added to give more information to the imputer, but we should experiment with other columns as well
columns_for_imputation = ['luse1', 'luse2', 'luse3', 'luse4', 'luse5', 'luse6', 'lusage']
imputer = IterativeImputer(max_iter=16, random_state=6)
df_imputed = pd.DataFrame(imputer.fit_transform(df[columns_for_imputation]), columns=columns_for_imputation)

df[['luse1', 'luse2', 'luse3', 'luse4', 'luse5', 'luse6']] = df_imputed[['luse1', 'luse2', 'luse3', 'luse4', 'luse5', 'luse6']]
df.isnull().sum()

In [ ]:
# We'll try to train an LSTM forecast the last month lusage value, based on the previous ones
# More exactly, we'll train the LSTM based on data from 2009 (6 months), 2010 (5 months) and 2011 (4 months)
# The target will be the last month of 2011.. In case we need to forecast future months, then we'll just
# predict the last month, shift data by one to the left and insert the new predicted month - then predict again

# Here we'll reshape the df so that we have a column with all lusage value
reshaped_data = []
grouped = df.groupby('hh_id')
additional_features = ['zipcode', 'mozip', 'children', 'owner', 'income', 'hhsize']

for _, group in grouped:
    lusage_2009 = group.iloc[0][['luse1', 'luse2', 'luse3', 'luse4', 'luse5', 'luse6']].values
    lusage_2010_2011 = group['lusage'].values
    other_features = group.iloc[0][additional_features].values
    reshaped_data.append(list(other_features) + [list(lusage_2009) + list(lusage_2010_2011)])

reshaped_df = pd.DataFrame(reshaped_data, columns=additional_features+['lusage_sequence'])
reshaped_df.head()

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# As mentioned above, we'll use the first 15 months to train (X_lusage) and the last month being the target (y)
# The static featurs (X_static), as they're the same we'll be fed separately in the model
X_static = reshaped_df[['zipcode', 'mozip', 'children', 'owner', 'income', 'hhsize']].values
X_lusage_full = np.array(reshaped_df['lusage_sequence'].tolist())
X_lusage = X_lusage_full[:, :-1]
y = X_lusage_full[:, -1]

# Normalizing the data
scaler_static = MinMaxScaler()
scaler_lusage = MinMaxScaler()
scaler_y = MinMaxScaler()

X_static_scaled = scaler_static.fit_transform(X_static)
X_lusage_scaled = np.array([scaler_lusage.fit_transform(x.reshape(-1, 1)).flatten() for x in X_lusage])
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

X_train_static, X_test_static, X_train_lusage, X_test_lusage, y_train, y_test = train_test_split(
    X_static_scaled, X_lusage_scaled, y_scaled, test_size=0.2, random_state=6)

# LSTM expects a tensor, so we'll reshape to have (nr_samples = households, timesteps = 15months, features = 1 lusage)
X_train_lusage = X_train_lusage.reshape((X_train_lusage.shape[0], X_train_lusage.shape[1], 1))
X_test_lusage = X_test_lusage.reshape((X_test_lusage.shape[0], X_test_lusage.shape[1], 1))

In [ ]:
from keras.layers import Input, LSTM, Dense, Concatenate
from keras.models import Model

lusage_input = Input(shape=(X_train_lusage.shape[1], 1)) # 15 timesteps per household with 1 feature
static_input = Input(shape=(6,)) # 6 static featurs

lstm_output = LSTM(64)(lusage_input)
static_output = Dense(16, activation='relu')(static_input)
combined = Concatenate()([lstm_output, static_output])
output = Dense(1)(combined) # predicting only the last month value, based on both static and lusage data

model = Model(inputs=[lusage_input, static_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit([X_train_lusage, X_train_static], y_train,  epochs=50, batch_size=16, validation_data=([X_test_lusage, X_test_static], y_test))

In [ ]:
y_pred_scaled = model.predict([X_test_lusage, X_test_static])
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_original = scaler_y.inverse_transform(y_test)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
plt.plot(y_test_original[100:200], label='Original', marker='o', color='blue')
plt.plot(y_pred[100:200], label='Predicted', marker='x', color='red')
plt.title('Original vs Predicted LUsage')
plt.xlabel('Households')
plt.ylabel('LUsage')
plt.legend()
plt.show()

In [ ]:
# Let's try to also train a new LSTM with new features (such as the trend)
# We fill the first 2 nan values with the previous existing ones, and also we dont take into account the 16th month
reshaped_df['trend'] = reshaped_df['lusage_sequence'].apply(lambda x: pd.Series(x[:-1]).rolling(window=3).mean().fillna(pd.Series(x)).tolist())
X_trend = np.array(reshaped_df['trend'].tolist())

scaler_trend = MinMaxScaler()
X_trend_scaled = np.array([scaler_lusage.fit_transform(x.reshape(-1, 1)).flatten() for x in X_trend])
X_lstm_input = np.stack([X_lusage, X_trend], axis=-1)

X_train_static, X_test_static, X_train_lusage, X_test_lusage, y_train, y_test = train_test_split(
    X_static_scaled, X_lstm_input, y_scaled, test_size=0.2, random_state=6)

# LSTM expects a tensor, so we'll reshape to have (nr_samples = households, timesteps = 15months, features = 2 lusage + trend)
X_train_lusage = X_train_lusage.reshape((X_train_lusage.shape[0], X_train_lusage.shape[1], 2))
X_test_lusage = X_test_lusage.reshape((X_test_lusage.shape[0], X_test_lusage.shape[1], 2))

# New LSTM model (TO-DO: make functions for this as it got really messy)
lusage_input = Input(shape=(X_train_lusage.shape[1], 2)) # 15 timesteps per household with 2 features now
static_input = Input(shape=(6,)) # 6 static featurs

lstm_output = LSTM(64)(lusage_input)
static_output = Dense(16, activation='relu')(static_input)
combined = Concatenate()([lstm_output, static_output])
output = Dense(1)(combined) # predicting only the last month value, based on both static and lusage data

model = Model(inputs=[lusage_input, static_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit([X_train_lusage, X_train_static], y_train,  epochs=50, batch_size=16, validation_data=([X_test_lusage, X_test_static], y_test))

In [ ]:
y_pred_trend = model.predict([X_test_lusage, X_test_static])
y_pred_trend = scaler_y.inverse_transform(y_pred_trend)
y_test_original = scaler_y.inverse_transform(y_test)

plt.figure(figsize=(6, 4))
plt.plot(y_test_original[100:200], label='Original', marker='o', color='blue')
plt.plot(y_pred_trend[100:200], label='Predicted', marker='x', color='red')
plt.title('Original vs Predicted (considering also the trend)')
plt.xlabel('Households')
plt.ylabel('LUsage')
plt.legend()
plt.show()
# Considering the trend the predictions are much much better, even visually (also loss: 0.0125 vs 0.001)